In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
working_dir = "../input"
# Any results you write to the current directory are saved as output.

from sklearn.model_selection import KFold
from scipy.stats import ttest_ind_from_stats
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import  OneHotEncoder
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC
import xgboost as xgb
from sklearn.decomposition import PCA

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import os
import json

['test', 'test_sentiment', 'train_sentiment', 'train']


In [55]:
def load_csv(path):
    csv_dataset = pd.read_csv(path,index_col="PetID")
    return csv_dataset

def map_name(name):
    name = str(name).lower()
    count = 0
    if name.find("name") >=0 or name.find("noname") >=0 or name.find("nameless") >=0 or name.find("no name") >=0 or name.find("unname") >=0 or name.find("to be name") >=0 or name.find("found") >= 0 or name.find("yet") >=0 or name.find("name less") >=0:
        return 0
    else:
        return 1
def transfor_features(csv_dataset):
    csv_dataset["Name"] = csv_dataset["Name"] .apply(map_name)
    csv_dataset.drop(["RescuerID","Description"],axis = 1, inplace=True)
    return csv_dataset

def get_max_abslute(sentences):
    max_ = 0
    real = 0
    for sentence in sentences:
        tmp = abs(sentence['sentiment']['score'])
        if tmp > max_:
            max_ = tmp
            real = sentence['sentiment']['score'] * sentence['sentiment']['magnitude']
    return real

def get_sentiment_dict(path):
    score_dict = {}
    score_magnitude = {}
    max_score = {}
    
    for file in os.listdir(path):
        if file.endswith(".json"):
            PetID = file.split(".")[0]
            with open(path + file,"r") as f:
                content = json.loads(f.read())

            max_score[PetID] = get_max_abslute(content['sentences'])
            score_magnitude[PetID] = content['documentSentiment']['score'] * content['documentSentiment']['magnitude']
            score_dict[PetID] = content['documentSentiment']['score']
    return score_dict,score_magnitude,max_score


def change_breed_train(csv_dataset):
    breed1_dict = {}
    all_breed1 = csv_dataset["Breed1"].unique().tolist()
    for name,group in csv_dataset.groupby("Breed1"):
        tmp_sum = 0
        for index, value in group.groupby("AdoptionSpeed").size().iteritems():
            tmp_sum += index * value / group.shape[0] 
        breed1_dict[name] = tmp_sum

    csv_dataset["Breed1"] = csv_dataset["Breed1"].replace(breed1_dict)


    breed2_dict = {}
    all_breed2 = csv_dataset["Breed2"].unique().tolist()
    for name,group in csv_dataset.groupby("Breed2"):
        tmp_sum = 0
        for index, value in group.groupby("AdoptionSpeed").size().iteritems():
            tmp_sum += index * value / group.shape[0] 
        breed2_dict[name] = tmp_sum


    csv_dataset["Breed2"] = csv_dataset["Breed2"].replace(breed2_dict)
    return breed1_dict,breed2_dict



def change_breed_test(csv_dataset,breed1_dict,breed2_dict):
    
    csv_dataset["Breed1"] = csv_dataset["Breed1"].replace(breed1_dict)
    csv_dataset["Breed2"] = csv_dataset["Breed2"].replace(breed2_dict)
    

def add_nlp_features(all_features,path):    
    score_dict,score_magnitude,max_score = get_sentiment_dict(path)
    all_features["score_magnitude"] = all_features.index
    all_features["score_magnitude"] = all_features["score_magnitude"].replace(score_magnitude)

    all_features["score"] = all_features.index
    all_features["score"] = all_features["score"].replace(score_dict)

    all_features["max_score"] = all_features.index
    all_features["max_score"] = all_features["max_score"].replace(max_score)
    
    
    def func(value):
        if type(value) == type("value"):
            return 0
        else:
            return value
    
    all_features["score_magnitude"] = all_features["score_magnitude"].apply(func)
    all_features["score"] = all_features["score"].apply(func)
    all_features["max_score"] = all_features["max_score"].apply(func)
    
    return all_features


def standard_and_dummy(all_features):
    ## 标准化数字特征
    numerical_features = ["Age","Breed1","Breed2","VideoAmt","PhotoAmt","Fee","Quantity","score_magnitude","score","max_score"]
    non_numerical_features = [feature for feature in all_features.columns if feature not in numerical_features]

    all_features[numerical_features] = all_features[numerical_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
    all_features = all_features.fillna(all_features.mean())
    
    ## 离散化类别特征
    all_features = pd.get_dummies(all_features, dummy_na=False, columns = non_numerical_features)
    return all_features

def prapare_dataset(train_path, test_path, train_nlp_path, test_nlp_path):
    train_dataset = load_csv(train_path)
    train_dataset["Name"].fillna(value = "noname", inplace = True)
    train_dataset = transfor_features(train_dataset)   
    
    ## 修改 breed 特征
    breed1_dict,breed2_dict = change_breed_train(train_dataset)
    
    
    test_dataset = load_csv(test_path)
    test_dataset["Name"].fillna(value = "noname", inplace = True)
    test_dataset = transfor_features(test_dataset)
    
    ## 修改 breed 特征
    change_breed_test(test_dataset,breed1_dict,breed2_dict)
    

    labels = train_dataset.iloc[:,-1]
    train_dataset = train_dataset.iloc[:,0:-1]
    
    train_features = add_nlp_features(train_dataset,train_nlp_path)
    test_features = add_nlp_features(test_dataset,test_nlp_path)

    all_features = pd.concat([train_features,test_features])
    all_features = standard_and_dummy(all_features)

    return all_features.loc[train_features.index.tolist(),:],all_features.loc[test_features.index.tolist(),:],labels
    

In [56]:
train_path = working_dir+"/train"+"/train.csv"
train_nlp_path = working_dir + "/train_sentiment/"
test_path = working_dir + "/test" + "/test.csv"
test_nlp_path = working_dir + "/test_sentiment/"

train_features, test_features, labels = prapare_dataset(train_path, test_path, train_nlp_path, test_nlp_path)

In [57]:
train_features.head()

Age    Breed1    Breed2  Quantity       Fee  VideoAmt  \
PetID                                                                   
86e1089a3 -0.421093 -0.045175 -0.024371 -0.390451  0.960741 -0.162955   
6296e909a -0.530696 -0.066649 -0.024371 -0.390451 -0.279382 -0.162955   
3422e4906 -0.530696  0.040329 -0.024371 -0.390451 -0.279382 -0.162955   
5842f1ff5 -0.366291  0.040329 -0.024371 -0.390451  1.580803 -0.162955   
850a43f90 -0.530696  0.040329 -0.024371 -0.390451 -0.279382 -0.162955   

           PhotoAmt  score_magnitude     score  max_score  ...  State_41330  \
PetID                                                      ...                
86e1089a3 -0.816653         0.163569  0.099727   0.418541  ...            0   
6296e909a -0.532363        -0.984042 -1.700960  -1.971336  ...            0   
3422e4906  0.889087         0.190258 -0.260411  -3.046781  ...            0   
5842f1ff5  1.173377         0.283668  2.260550   0.824820  ...            0   
850a43f90 -0.248073         2.165217  1.180138   0.418541  ...            0   

           State_41332  State_41335  State_41336  State_41342  State_41345  \
PetID                                                                        
86e1089a3            0            0            0            0            0   
6296e909a            0            0            0            0            0   
3422e4906            0            0            0            0            0   
5842f1ff5            0            0            0            0            0   
850a43f90            0            0            0            0            0   

           State_41361  State_41367  State_41401  State_41415  
PetID                                                          
86e1089a3            0            0            0            0  
6296e909a            0            0            1            0  
3422e4906            0            0            0            0  
5842f1ff5            0            0            1            0  
850a43f90            0            0            0            0  

[5 rows x 70 columns]

In [58]:
train_features.shape, test_features.shape, labels.shape

((14993, 70), (3948, 70), (14993,))

In [59]:
# def tune_parameters(training_dataset,labels):
    
#     tuned_parameters = {
#     "estimator__solver": ["lbfgs","sag"],
#     "estimator__C": [0.1, 1,10],
#     "estimator__max_iter": [500,1000],
#     "estimator__fit_intercept": [True,False],
#     }

#     scores = ['accuracy']

#     # 将数据集分成训练集和测试集
                        
#     for score in scores:
#         print("# Tuning hyper-parameters for %s\n" % score)

#         clf = GridSearchCV(OneVsRestClassifier(LogisticRegression()), tuned_parameters, cv = 5,
#                            scoring='%s' % score)

#         clf.fit(training_dataset, labels)

#         print("Best parameters set found on development set:\n")
#         #输出最优的模型参数
#         print(clf.best_params_)
        
#     return clf

# model = tune_parameters(train_features,labels)

In [1]:
X_train, X_test, y_train, y_test = train_test_split(
    train_features, labels, test_size=0.2, random_state=0)

params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',  # 多分类的问题
    'num_class': 5,               # 类别数，与 multisoftmax 并用
    'gamma': 0.1,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth': 10,               # 构建树的深度，越大越容易过拟合
    'lambda': 2,                   # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample': 0.8,              # 随机采样训练样本
    'colsample_bytree': 0.8,       # 生成树时进行的列采样
    'min_child_weight': 3,
    'silent': 1,                   # 设置成1则没有运行信息输出，最好是设置为0.
    'eta': 0.001,                  # 学习率
    'seed': 1000,
    'nthread': 4,                  # cpu 线程数
}

clf = xgb.XGBModel(**params)
model = clf.fit(X_train,y_train)

NameError: name 'train_test_split' is not defined

In [63]:
pred = model.predict(X_test)
print(sum(pred == y_test)/len(pred))
print(classification_report(pred,y_test))

0.42880960320106704
              precision    recall  f1-score   support

         0.0       0.07      0.83      0.12         6
         1.0       0.37      0.41      0.39       581
         2.0       0.40      0.35      0.38       915
         3.0       0.26      0.41      0.32       414
         4.0       0.67      0.51      0.58      1083

   micro avg       0.43      0.43      0.43      2999
   macro avg       0.35      0.50      0.36      2999
weighted avg       0.47      0.43      0.44      2999



In [64]:
pred_labels = model.predict(test_features)
output = pd.DataFrame(
    data = { "PetID": test_features.index,
            "AdoptionSpeed": pred_labels  
            },
    dtype = np.int32
)
output.head(10)

PetID  AdoptionSpeed
0  378fcc4fc              2
1  73c10e136              4
2  72000c4c5              4
3  e147a4b9f              2
4  43fbba852              4
5  77a490ec9              4
6  28c4b1b13              4
7  d1eada628              4
8  d134dec34              4
9  bcd464bb8              1

In [ ]:
output.to_csv("submission.csv",index = False)